In [ ]:
# O mesmo que o script2, mas com a opção de adicionar uma blacklist de codigos brite (kegg)

In [1]:
import pandas as pd
from pandas import ExcelWriter
import tkinter as tk
from tkinter import filedialog
import time
import requests
import json
from IPython.display import clear_output

In [2]:
# Abre o ficheiro .tsv
root = tk.Tk()
root.withdraw()
print ('A abrir ficheiro...')
file = filedialog.askopenfilename(filetypes = [("TSV","*.tsv")])

A abrir ficheiro...


In [3]:
print ('A iniciar...')
start_time = time.time()
h = []
with open(file, 'r') as f:
    x = f.read()
    x = x.split('\n')
    for y in x:
        h.append(y)

A iniciar...


In [4]:
h[len(h)-2]
new = [h[len(h)-2]]
x = 0
while x < len(h)-2:
    new.append(h[x])
    x = x+1
new = "\n".join(new)

In [5]:
f = open(file[:-4]+'_int.tsv', 'w')
f.write(new)
f.close()

In [6]:
file = file[:-4]+'_int.tsv'

In [7]:
# Abre o ficheiro .tsv no pandas
df = pd.read_csv(file, sep='\t')

In [8]:
#Anota os compostos do KEGG
def kegg(c):
    k = requests.get('http://rest.kegg.jp/get/' + c).text
    if 'BRITE' not in k:
        if 'LIPIDMAPS:' in k:
            k = k.splitlines()
            for x in k:
                if 'LIPIDMAPS:' in x:
                    x = x[len('            LIPIDMAPS: '):]
                    a = lipidmaps(x)
                    mm = a[0]
                    cc = a[1]
                    ss = a[2]
                    tt = a[3]
                    return (mm, cc, ss, tt)
                else:
                    continue
        else:
            return None
    else:
        major=[]
        clas=[]
        second=[]
        third=[]
        k = k.split('BRIT', 1)[1]
        k = k.split('DBLINKS', 1)[0]
        k = k.splitlines()
        l = []
        for x in k:
            if x.startswith('E       '):
                l.append('            ' + x[len('E       '):])
            else:
                l.append(x)
        l = "\n".join(l)
        l = l.split('           ')[1:]
        p = []
        for x in l:
            if not x.startswith('  '):
                x = ('??'+x)
                p.append(x)
            else:
                p.append(x)
        p = "".join(p)
        p = p.split('??')
        for o in range(10):
            for x in p:
                if any([y in x for y in blacklist]):
                    p.remove(x)
        p = "\n".join(p)
        p = p.splitlines()
        for x in p:
            if x.startswith(' ') and not x.startswith('  '):
                major.append(x[1:])
            elif x.startswith('  ') and not x.startswith('   '):
                clas.append(x[2:])
            elif x.startswith('   ') and not x.startswith('    '):
                second.append(x[3:])
            elif x.startswith('    ') and not x.startswith('     '):
                third.append(x[4:])
        mm = "#".join(major)
        cc = "#".join(clas)
        ss = "#".join(second)
        tt = "#".join(third)
        return (mm, cc, ss, tt)

In [9]:
# Anota os compostos do LipidMaps
def lipidmaps(x):
    f = requests.get('http://www.lipidmaps.org/rest/compound/lm_id/' + x + '/all/json')
    s = json.loads(f.text)
    if f.text == '[]':
        mm = 'null'
        cc = 'null'
        ss = 'null'
        tt = 'null'
        return (mm, cc, ss, tt)
    if s['core'] != None:
        mm = 'Lipids [LM]'
        cc = s['core']
    else:
        mm = 'Lipids [LM]'
        cc = 'null'
    if s['main_class'] != None:
        ss = s['main_class']
    else:
        ss = 'null'
    if s['sub_class'] != None:
        tt = s['sub_class']
    else:
        tt = 'null'
    return (mm, cc, ss, tt)

In [10]:
# Diz se o composto está presente em plantas (Plantae), através da base de dados knapsack
def knap(c):
    m = requests.get('http://rest.kegg.jp/get/' + c).text
    if 'KNApSAcK' in m:
        m = m.splitlines()
        for x in m:
            if x.startswith('            KNApSAcK: '):
                r = requests.post('http://kanaya.naist.jp/knapsack_jsp/information.jsp?sname=C_ID&word=' + (x[len('            KNApSAcK: '):]))
                if 'Plantae' in r.text:
                    return 'Plantae'
                else:
                    return ''
    else:
        return ''

In [11]:
# Anota as 4 classes, dependendo se é código kegg, ou lipidmaps
def anotar(x):
    global count
    splittext = x.split('#')
    major=[]
    clas=[]
    second=[]
    third=[]
    for y in splittext:
        if y.startswith('C'):
            a = kegg(y)
            if a != None :
                major.append(a[0])
                clas.append(a[1])
                second.append(a[2])
                third.append(a[3])
            else:
                continue
        elif y.startswith('L'):
            a = lipidmaps(y)
            major.append(a[0])
            clas.append(a[1])
            second.append(a[2])
            third.append(a[3])
        else:
            continue
    clear_output()
    count=count+1
    print (str(round((1-((total-count)/total))*100)) + "% concluído")
    
    major = "#".join(set(major))
    clas = "#".join(set(clas))
    second = "#".join(set(second))
    third = "#".join(set(third))
    
    return pd.Series([major,clas,second,third], index=['Major Class', 'Class', 'Secondary Class', 'Tertiary Class'])

In [12]:
# Chama a função knap, para os diferentes compostos
def knapsack(x):
    global count
    splittext = x.split('#')
    knapsack=[]
    for y in splittext:
        if y.startswith('C'):
            knapsack.append(knap(y))
        else:
            continue
    clear_output()
    count=count+1
    print (str(round((1-((total-count)/total))*100)) + "% concluído")
    return "".join(set(knapsack))

In [13]:
# Cria BlackList com base nos códigos br:####### presentes no ficheiro blacklist.txt
blacklist=[]
with open('blacklist.txt', 'r') as f:
    x = f.read()
    x = x.split('\n')
    for y in x:
        blacklist.append(y)

In [14]:
# Dicionário de conversão de HMDB para KEGG
h = requests.post('http://rest.genome.jp/link/compound/hmdb').text.splitlines()
dic = {}
for x in h:
    dic[x[5:14]] = x[19:25]

In [15]:
# Dicionários de conversão de KEGG para Lipid Maps e Lipid Maps para KEGG
h = requests.get('http://rest.genome.jp/link/compound/lipidmaps').text
dicl = {} # Dicionário kegg2lipidmaps
dicl2 = {} # Dicionário lipidmaps2kegg
h = h.replace("lipidmaps:", "")
h = h.replace("cpd:", "")
h = h.replace("\tequivalent", "")
h = h.splitlines()
for x in h:
    dicl[x.split("\t",1)[1]] = x.split("\t",1)[0]
    dicl2[x.split("\t",1)[0]] = x.split("\t",1)[1]

In [16]:
# Criar coluna de convertidos HMDB to KEGG
def hmdb2kegg(x):
    splittext = x.split('#')
    convert=[]
    for y in splittext:
        if y in dic:
            convert.append(dic[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [17]:
# Criar coluna de convertidos KEGG para Lipid Maps
def kegg2lipidmaps(x):
    splittext = x.split('#')
    convert=[]
    for y in splittext:
        if y in dicl:
            convert.append(dicl[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [ ]:
# Criar coluna de convertidos KEGG para Lipid Maps
def lipidmaps2kegg(x):
    splittext = x.split('#')
    convert=[]
    for y in splittext:
        if y in dicl2:
            convert.append(dicl2[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [18]:
# Retira as colunas 9 a 20, que não são necessárias
# Exprimentei df.drop(df.columns[['nome', 'nome 2', ..., 'nome n']], axis=1, inplace=True) mas não resulta
df.drop(df.columns[[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], axis=1, inplace=True)

In [19]:
# Contagens para os n% concluido.
total = len(df['raw_mass']) * 2
count = 0

In [20]:
# Converte os códigos HMDB para KEGG, se possivél, através do dicionário "dic"
df['HMDB_2_KEGG'] = df['KEGG_cid'].apply(hmdb2kegg)
# Converte os códigos KEGG para Lipid Maps, se possível, através do dicionário "dicl"
df['KEGG_2_LipidMaps'] = df['HMDB_2_KEGG'].apply(kegg2lipidmaps)
# Converte os códigos Lipid Maps para KEGG, se possível, através do dicionário "dicl2"
df['LipidMaps_2_KEGG'] = df['HMDB_2_KEGG'].apply(lipidmaps2kegg)

In [21]:
# Aplica as funções de análise de códigos KEGG e Lipid MAPS, e retorna a anotação pretendida
df = pd.concat([df,df['KEGG_2_LipidMaps'].apply(anotar)], axis=1) # Anota os compostos nas 4 classes pretendidas.
df['KNApSAcK'] = df['LipidMaps_2_KEGG'].apply(knapsack) # Apenas os codigos KEGG dão acesso ao knapsak, como tal a pesquiza no knapsack é feita nesta coluna.

100% concluído


In [40]:
# Guarda o novo dataframe para um ficheiro EXCEL (.xlsx) com o mesmo nome do ficheiro .tsv, acrecentado '_no-drugs' ao nome
writer = ExcelWriter(file[:-8]+'_no-drugs.xlsx')
df.to_excel(writer, header=True, index=False, sheet_name='No Drugs')

In [41]:
# Faz as contagens das diferentes classes de compostos
col = ['Major Class', 'Class', 'Secondary Class', 'Tertiary Class']
for x in col:
    lista = df[x].tolist()
    lista = "#".join(lista)
    lista2 = []
    for y in lista.split("#"):
        lista2.append(y)
    new_df = pd.Series(lista2)
    new_df = new_df.to_frame(x)
    major = new_df.groupby(x).size()
    major = major.to_frame('Contagem')
    major.to_excel(writer, header=True, index=True, sheet_name=x)
writer.save()

In [42]:
# Célula de saida
clear_output()
elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
print ("Concluído em: " + "%02d:%02d" % (m, s) + 's')

Concluído em: 25:25s
